# Import

First, import the Python modules necessary for this notebook to run. These can be installed using, for instance, `pip` or `conda`. 

In [1]:
import numpy as np # Handles maths
import pandas as pd # Good for tables of data
import matplotlib.pyplot as plt # Handles graphing
import requests # Downloads webpages
from bs4 import BeautifulSoup # For parsing webpages
from selenium import webdriver 
from selenium.webdriver.common.action_chains import ActionChains
import os, sys
import time
import subprocess
import re
import string

import scrape_EPBC
%load_ext autoreload
%autoreload 2

# Initialise

In [2]:
# stored_table = pd.read_csv('/home/student.unimelb.edu.au/shorte1/Documents/ACF_consulting/EPBC_notices_test.csv')
# stored_table[stored_table['Reference Number'] == '2019/8527']
# stored_table = stored_table.drop(1539, axis=0).reset_index(drop=True)
# stored_table.to_csv('/home/student.unimelb.edu.au/shorte1/Documents/ACF_consulting/EPBC_notices_test.csv', index=False, header=True)

In [3]:
url = "http://epbcnotices.environment.gov.au/publicnoticesreferrals"

In [4]:
options = webdriver.ChromeOptions()
# options.add_argument('--headless') # Comment out to see the actions on website
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument("--start-maximized")

base_dir = '/home/student.unimelb.edu.au/shorte1/Documents/ACF_consulting/' 
sub_dir = 'files_test'
files_dir = base_dir + sub_dir 

options.add_experimental_option("prefs", {
  "download.default_directory": files_dir,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True,
  "plugins.always_open_pdf_externally": True
})

driver = webdriver.Chrome('/usr/bin/chromedriver', options=options)
driver.get(url);
time.sleep(4)

def clean_columns(table):
    name_dict = {}
    clean_str = '  . Activate to sort in descending order'
    for col in range(len(table.columns)): 
        name_dict[table.columns[col]] = table.columns[col].replace(clean_str, '')
    return table.rename(name_dict, axis='columns')

# Define Page Scraper


Two modes - download or update mode.
If in download mode, search forward, skipping files that already exist 
in table.
If in update mode, search forward, but assume new entries to website
appear first, so stop when a certain number of matches have occured. 

In [5]:
%pdb

Automatic pdb calling has been turned ON


# Scrape Website

In [ ]:
for i in range(1,4):
    
    loading = True
    attempts = 0
    while loading:
        if attempts > 30:
            raise RuntimeError('Could not load website')
        try:
            time.sleep(2)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source)
            table = pd.read_html(soup.prettify())[0]
            if len(table) == 30:
                loading = False
        except:
            attempts += 1

    table = clean_columns(table)
    table['Attachments'] = ['TBD']*30
    table['Download'] = ['TBD']*30
    table['Download Folder'] = ['TBD']*30    
    table['Date of notice'] = pd.to_datetime(
        table['Date of notice'], dayfirst=True)
    table.drop(labels='Actions', axis=1, inplace=True)  
    
    try:
        stored_table = pd.read_csv(base_dir + '/EPBC_notices_test.csv')
        stored_table['Date of notice'] = pd.to_datetime(
            stored_table['Date of notice'], dayfirst=True)
        label_list = ['Attachments', 'Download', 'Download Folder']
        shared = pd.merge(
            table.drop(labels=label_list, axis=1), 
            stored_table.drop(labels=label_list, axis=1), 
            how='left', indicator='Exist')
        shared['Exist'] = np.where(shared.Exist == 'both', True, False)
        exist = shared['Exist']
        del shared
    except:
        stored_table = table.iloc[0:0]
        stored_table['Date of notice'] = pd.to_datetime(
            stored_table['Date of notice'], dayfirst=True)
        exist = [False]*30
        exist = pd.Series(exist,name='Exist')
     
    if np.any(~exist):
        scrape_EPBC.scrape_page(
            driver, i, table, stored_table, exist, base_dir, files_dir)
    
    try:
        next_button = driver.find_elements_by_xpath(
            '//a[@href="#" and @data-page="' + str(i+1) + '"]'
        )[1]
        ActionChains(driver).move_to_element(next_button).perform()
        next_button.click()
    except:
        print('Quitting.')
    
    del table, stored_table
    
driver.quit()

Still Downloading. Please Wait.
Still Downloading. Please Wait.
> /home/student.unimelb.edu.au/shorte1/Documents/ACF_consulting/scrape_EPBC.py(144)scrape_page()
    142         if len(file_links) >= 17:
    143             import pdb; pdb.set_trace()
--> 144         [num_files, file_names] = [[], []]
    145         if not file_links:
    146             num_files.append(0)

